# Bug of a tool not pointing to the center

Despite now the tool seem to pick up tips, it still clearly not pointing to the center of the tube slots in 50 mL or eppendorf tube racks.

Here I try to debug the problem

In [1]:
import tools
import cartesian
import racks
import calibration
import samples
import json
import importlib

# Reproducing the problem

In [2]:
ar = cartesian.arnie(cartesian_port="COM10", docker_port="COM6")

In [48]:
ar.home()

In [4]:
stp = tools.stationary_touch_probe(ar)

ERROR:root:matchPortsWithDevices(): could not execute finiding device in list.
ERROR:root:matchPortsWithDevices(): Provided patterns_list: ['stationary touch probe']
ERROR:root:None


In [6]:
p1000 = tools.pipettor.getTool(robot=ar, tool_name='p1000_tool')

In [5]:
p1000 = tools.pipettor(robot=ar, tool_name='p1000_tool')

In [6]:
xt, yt, zt = calibration.calibrateTool(p1000, stp)

In [7]:
ar.move(z=400)

In [8]:
p1000_tip_rack = racks.rack(rack_name='p1000_tips_1', rack_type='p1000_tips')

In [9]:
p1000.pickUpTip(p1000_tip_rack, 0, 0, raise_z=400)

In [10]:
ar.move(z=300)

In [11]:
ep1 = racks.rack(rack_name='eppendorf_1', rack_type='eppendorf')

In [24]:
#w1x, w1y = ep1.calcWellXY(7, 3)
w1x, w1y, w1z = ep1.calcWorkingPosition(7, 3, p1000)

In [25]:
ar.move(x=w1x, y=w1y)

In [14]:
ar.move(z=367)

In [27]:
p1000.dropOffTipToPosition(p1000_tip_rack, 0, 0, raise_z=300, dropoff_dz=-40)

In [28]:
p1000.returnTool()

## Analyzing positions

In [26]:
# This supposed to be position 9, 7
ar.getPosition()

(150.8, 436.75, 367.0)

In [16]:
ep1.rack_data

{'name': 'eppendorf_1',
 'n_x': 0,
 'n_y': 3,
 'type': 'eppendorf',
 'position': [91.35, 395.29999999999995, 508.9],
 'pos_stalagmyte': [92.85, 66.15, 490.7]}

In [15]:
p1000.getStalagmyteCoord()

(92.8, 66.6, 351.55)

In [18]:
x_dist_cntr = 59.5
y_dist_cntr = 41.0

In [20]:
x_cntr = ep1.rack_data['position'][0]
y_cntr = ep1.rack_data['position'][1]
x_cntr, y_cntr

(91.35, 395.29999999999995)

In [21]:
x_coord_73 = x_cntr + x_dist_cntr
y_coord_73 = y_cntr + y_dist_cntr

In [22]:
x_coord_73, y_coord_73

(150.85, 436.29999999999995)

# Re-calibrating racks

In [49]:
tp = tools.mobile_touch_probe.getTool(ar)

In [50]:
x, y, z = calibration.calibrateStationaryProbe(tp, stp)

In [51]:
xr, yr, zr = calibration.calibrateRack(tp, p1000_tip_rack)

In [52]:
ar.move(z=300)

In [53]:
xr, yr, zr = calibration.calibrateRack(tp, ep1)

In [54]:
tp.returnTool()

In [35]:
ep1.rack_data

{'name': 'eppendorf_1',
 'n_x': 0,
 'n_y': 3,
 'type': 'eppendorf',
 'position': [90.75, 395.04999999999995, 509.45],
 'pos_stalagmyte': [92.69999999999999, 66.5, 491.3]}

In [36]:
p1000_tip_rack.rack_data

{'name': 'p1000_tips_1',
 'n_x': 0,
 'n_y': 2,
 'type': 'p1000_tips',
 'position': [91.14, 285.35, 528.5],
 'pos_stalagmyte': [92.69999999999999, 66.5, 491.3]}

In [55]:
p1000 = tools.pipettor.getTool(robot=ar, tool_name='p1000_tool')

In [56]:
xt, yt, zt = calibration.calibrateTool(p1000, stp)

In [57]:
p1000.pickUpTip(p1000_tip_rack, 0, 0, raise_z=400)

In [58]:
ar.move(z=300)

In [59]:
w1x, w1y, w1z = ep1.calcWorkingPosition(7, 3, p1000)

In [60]:
ar.move(x=w1x, y=w1y)

In [61]:
ar.move(z=367)

In [62]:
p1000.dropOffTipToPosition(p1000_tip_rack, 0, 0, raise_z=300, dropoff_dz=-40)

In [63]:
p1000.returnTool()

In [77]:
ar.home()

# Gripper

In [65]:
gr = tools.mobile_gripper.getTool(robot=ar)

In [66]:
calibration.calibrateToolCustomPoints(tool=gr, stationary_probe=stp)

(93.93, 67.4, 516.05)

In [67]:
ar.move(z=300)

In [68]:
w1x, w1y, w1z = ep1.calcWorkingPosition(7, 3, gr)

In [69]:
ar.move(x=w1x, y=w1y)

In [70]:
ar.move(z=w1z)

In [73]:
gr.operateGripper(100)

In [75]:
gr.operateGripper(0)

In [76]:
gr.returnTool()